In [ ]:
# Copyright 2019 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
# Install Pipeline SDK - This only needs to be ran once in the enviroment. 
!python3 -m pip install 'kfp>=0.1.31' --quiet

# KubeFlow Pipelines - Creating an environment validation step using KFP diagnose_me libraries 

This notebook creates a parametrized environment validation step that allows users to specify Google Cloud APIs that are needed for pipeline execution and validates if they are enabled. 


In [ ]:
def verify_gcp_credentials() -> bool:
    """ This pipeline steps verifies if gcp credentials are configured correctly.

    Raises:
        RuntimeError: If gcp credentials are not configured correctly
    """

    # Install pip3 and kfp on this image
    import subprocess
    subprocess.run(
      ['curl', 'https://bootstrap.pypa.io/get-pip.py', '-o', 'get-pip.py'],
      capture_output=True)
    subprocess.run(['apt-get', 'install', 'python3-distutils', '--yes'],
                 capture_output=True)
    subprocess.run(['python3', 'get-pip.py'], capture_output=True)
    subprocess.run(['python3', '-m', 'pip', 'install', 'kfp', '--quiet'],
                 capture_output=True)

    import sys
    from typing import List, Text
    import os
    from kfp.cli.diagnose_me import gcp

    # Get the project ID to construct an API enable link if needed
    project_config = gcp.get_gcp_configuration(
      gcp.Commands.GET_GCLOUD_DEFAULT, human_readable=False)
    project_id = ''
    if not project_config.has_error:
        project_id = project_config.parsed_output['core']['project']
        print('GCP credentials are configured with access to project: %s ...' %
              (project_id))
        return True

    raise RuntimeError(
      'Project configuration is not accessible with error  %s\n' %
      (project_config.stderr) + 'Follow the instructions at\n' +
      'https://github.com/kubeflow/pipelines/tree/master/samples/core/parameterized_tfx_oss#permission\n'
      + 'to verify you have configured the required gcp secret.')

In [ ]:
def verfiy_gcp_apis(target_apis:str)-> bool:
    """ This pipeline steps verifies if specified APIs are enabled under the gcp project.
    
    
    Args: 
        target_apis: comma separated name of the apis
    
    Raises:
        RuntimeError: If gcp secret is not configured correctly, or service account does not 
        have proper previliage to acccess the API status. 
    """
    
    # Install pip3 and kfp on this image 
    import subprocess
    subprocess.run(['curl','https://bootstrap.pypa.io/get-pip.py','-o','get-pip.py'], capture_output=True)
    subprocess.run(['apt-get', 'install', 'python3-distutils','--yes'], capture_output=True)
    subprocess.run(['python3', 'get-pip.py'], capture_output=True)
    subprocess.run(['python3', '-m','pip','install','kfp', '--quiet'], capture_output=True)
    
    
    import sys
    from typing import List, Text 
    import os
    from kfp.cli.diagnose_me import gcp
    
    # Get the project ID to construct an API enable link if needed
    project_config = gcp.get_gcp_configuration(gcp.Commands.GET_GCLOUD_DEFAULT,human_readable=False)
    project_id = ''   
    if not project_config.has_error:
        project_id = project_config.parsed_output['core']['project']
        print('Verifying APIs in project %s ...' % (project_id))
    else: 
        project_id = '<YOUR-GCP-PROJECT-ID>'
        raise RuntimeError('Could not retrieve project ID with error  %s' % (project_config.stderr))
        
    # Get the status of GCP APIs and add the results to a dictionary
    api_config_results = gcp.get_gcp_configuration(
        gcp.Commands.GET_APIS)
    
    api_status = {}
    
    if api_config_results.has_error:
        raise RuntimeError('could not retrieve API status with error: %s' %(api_config_results.stderr))
    
    for item in api_config_results.parsed_output:
        api_status[item['config']['name']] =  item['state']
        # printing the results in stdout for logging purposes 
        print('%s %s' % (item['config']['name'], item['state']))
    

    # Check if target apis are enabled 
    api_check_results = True
    error_list = []
    for api in target_apis.replace(' ','').split(','): 
        if 'ENABLED'!= api_status.get(api, 'DISABLED'):
            api_check_results = False
            error_list.append('API \"%s\" is not enabled. To enable this api go to https://pantheon.corp.google.com/apis/library/%s?project=%s' %(api,api,project_id))
            
    if api_check_results:
        return True
    else:
        raise RuntimeError('Required APIs are not enabled:\n'+ '\n'.join(error_list))

In [ ]:
import kfp.components as comp
verify_gcp_credentials_op = comp.func_to_container_op(
    verify_gcp_credentials, base_image='google/cloud-sdk:latest')
verify_gcp_apis_op = comp.func_to_container_op(
    verfiy_gcp_apis, base_image='google/cloud-sdk:latest')

In [ ]:
from kfp.gcp import use_gcp_secret
from kfp import dsl

@dsl.pipeline(
    name='Verify KFP Env',
    description="""
    Verfies if env is configured properly by specifically verifying
    - Credentials are set correctly
    - The specified APIs are enabled in the project. To learn more about
    available APIs go to https://pantheon.corp.google.com/apis/library/."""
)
def verify_gcp_kfp_env(
    target_apis='stackdriver.googleapis.com, storage-api.googleapis.com, '
                'bigquery.googleapis.com, dataflow.googleapis.com'
):
    """A sample pipeline that verifies KFP redentials and API"""
    task1 = verify_gcp_credentials_op().apply(use_gcp_secret('user-gcp-sa'))
    task2 = verify_gcp_apis_op(target_apis).apply(use_gcp_secret('user-gcp-sa'))
    task2.after(task1)

In [ ]:
import kfp
client = kfp.Client(host='d9f20dcfd673558-dot-us-central1.notebooks.googleusercontent.com')

In [ ]:
client.create_run_from_pipeline_func(verify_gcp_kfp_env, arguments={})

In [ ]:
kfp.compiler.Compiler().compile(verify_gcp_kfp_env,'kfp_env_validation.yaml')